In [ ]:
!pip3 install us

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# TO-Do use the output from model
import pandas as pd
import numpy as np
import us
import re

from city_to_state import city_to_state_dict
from geopy import geocoders 

In [ ]:
data = pd.read_csv("atweets_complete_labeled.csv")
data.shape #15073*36

(15073, 36)

In [ ]:
states_lat = pd.read_csv('states_latandlong.csv')
states_lat['abbr'] = ''
# create distances dataframe
for i in range(states_lat.shape[0]):
  states_lat['abbr'][i] = us.states.lookup(str(states_lat['state'][i])).abbr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
states_lat

,state,latitude,longitude,abbr
0,Alabama,32.7794,-86.8287,AL
1,Alaska,64.0685,-152.2782,AK
2,Arizona,34.2744,-111.6602,AZ
3,Arkansas,34.8938,-92.4426,AR
4,California,37.1841,-119.4696,CA
5,Colorado,38.9972,-105.5478,CO
6,Connecticut,41.6219,-72.7273,CT
7,Delaware,38.9896,-75.5050,DE
8,District of Columbia,38.9101,-77.0147,DC
9,Florida,28.6305,-82.4497,FL


In [ ]:
data.user_location.sample(20)

273               San Francisco, CA
13412                           NaN
8965                            NaN
1474       18 // black&white // bi 
6649                            NaN
3876                London, England
7238           Mississauga, Ontario
11067         San Diego, California
6666     the marshes in the valley.
8565                Tokyo-to, Japan
1461                            NaN
8617                            NaN
2782                London, England
11134     xwesam - Roberts Creek,BC
11498           Karachi Abbasi home
11171     Kelowna, British Columbia
10857                Nangal, Punjab
11144                           NaN
13125                           NaN
13266                           NaN
Name: user_location, dtype: object

In [ ]:
two_word_states = [
'new york',
'new hampshire',
'new jersey',
'new mexico',
'north dakota',
'south dakota',
'north carolina',
'south carolina',
'rhode island',
'west virginia'
]

In [ ]:
# clean user location 
# https://medium.com/swlh/extracting-location-data-from-twitter-54c837144038
data['abbr'] = ''
def get_state_abbr(x, ind):
  if type(x[ind]) == str and pd.isna(x[ind]) == False:
    if re.match('({})'.format("|".join(two_word_states)), x[ind].lower()):
      tokens = [re.match('({})'.format("|".join(two_word_states)), x[ind].lower()).group(0)]
    elif re.match('({})'.format("|".join(city_to_state_dict.keys()).lower()), x[ind].lower()):
      k = re.match('({})'.format("|".join(city_to_state_dict.keys()).lower()), x[ind].lower()).group(0)
      tokens = [city_to_state_dict.get(k.title(), np.nan)]
    else:
      tokens = [j for j in re.split("\s|,", x[ind]) if j not in ['in', 'la', 'me', 'oh', 'or']]
    for i in tokens:
      if re.match('\w+', str(i)):
        if us.states.lookup(str(i)):
          data['abbr'][ind] = us.states.lookup(str(i)).abbr
  else:
    data['abbr'][ind] = float('nan')

In [ ]:
for ind in range(data['user_location'].shape[0]):
  get_state_abbr(data['user_location'], ind)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data[['user_location','abbr']][:100]

,user_location,abbr
0,"Washington, DC",IL
1,"Springfield, Mo",MA
2,"Maryland, USA",MD
3,NaN,NaN
4,Your mom's bedroom,
...,...,...
95,"she/her 28 New Jersey, USA",MS
96,NaN,NaN
97,"Portland, OR",OR
98,"New Jersey, USA",NJ


In [ ]:
comb = pd.merge(data,states_lat, how='left', on='abbr')

In [ ]:
comb.sample(100)

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,is_misinfo,abbr,state,latitude,longitude
7612,NaN,Thu Apr 29 03:36:32 +0000 2021,DrLiMengYan CCPVirus Covid19 UnrestrictedBiowe...,NaN,https://www.gnews.org/1148627/,6.0,1387611732332343296,NaN,NaN,NaN,...,edendomain,18677.0,NaN,NaN,False,1,,NaN,NaN,NaN
8210,NaN,Tue Apr 27 21:43:50 +0000 2021,COVID19 CovidVaccine GetVaccinated,NaN,NaN,1.0,1387160587621240833,NaN,NaN,NaN,...,HumphriesDJA,686.0,NaN,NaN,False,0,,NaN,NaN,NaN
14269,NaN,NaN,NaN,NaN,NaN,NaN,1389694639880474625,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
6696,NaN,Tue Mar 16 03:02:48 +0000 2021,bloodclot StaySafe pfizer,NaN,NaN,0.0,1371658177780011009,NaN,NaN,NaN,...,Rive_Dopez,2035.0,NaN,NaN,False,1,,NaN,NaN,NaN
2489,NaN,Tue Jan 05 22:45:52 +0000 2021,NaN,NaN,NaN,6.0,1346588754270674946,NaN,NaN,NaN,...,Nakotablackelkn,3204.0,NaN,NaN,False,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12647,NaN,NaN,NaN,NaN,NaN,NaN,1342191221415022593,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1773,NaN,Wed Jan 06 19:43:02 +0000 2021,NaN,NaN,NaN,0.0,1346905130885652489,NaN,NaN,NaN,...,martiecee,16537.0,NaN,NaN,False,1,,NaN,NaN,NaN
6318,NaN,Fri Mar 19 16:47:51 +0000 2021,vaccinated,NaN,https://www.theatlantic.com/science/archive/20...,0.0,1372952970292293634,NaN,NaN,NaN,...,MichaelSmith2nd,6441.0,NaN,NaN,False,0,CA,California,37.1841,-119.4696
1766,NaN,Wed Jan 06 23:20:20 +0000 2021,NaN,NaN,https://bit.ly/35gMnVP,15.0,1346959817697615872,NaN,NaN,NaN,...,standardnews,422949.0,NaN,http://www.standard.co.uk,True,0,,NaN,NaN,NaN


In [ ]:
sum(comb['abbr'].isna())

5922

In [ ]:
sum(comb['latitude'].isna())

10723

In [ ]:
comb[(comb['latitude'].isna()==True) & (comb['abbr'].isna()==False)][['user_location','abbr','latitude','longitude']]

,user_location,abbr,latitude,longitude
4,Your mom's bedroom,,NaN,NaN
5,United States Of America,,NaN,NaN
7,Virgo Supercluster,,NaN,NaN
9,United States Of America,,NaN,NaN
11,Toronto,,NaN,NaN
...,...,...,...,...
12387,SHINee,,NaN,NaN
12393,South Africa,,NaN,NaN
12394,"Stirling, Scotland",,NaN,NaN
12397,Brushing my teeth in the hall,,NaN,NaN


In [ ]:
comb.columns

Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'quote_id', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_id', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified',
       'is_misinfo', 'abbr', 'state', 'latitude', 'longitude'],
      dtype='object')

In [ ]:
comb.shape

(15073, 40)

# Output

In [ ]:
comb.to_csv('lat_long.csv', index=False)

In [ ]:
# if there is some missing data, it will report the lat and long must be numberic
comb_complete = comb[(comb['abbr'].isna()==False) & (comb['state'].isna()==False)& (comb['abbr'] !='')]
comb_complete = pd.DataFrame(comb_complete.reset_index())
comb_complete.to_csv('lat_long_complete.csv', index=False)

In [ ]:
comb_complete.columns

Index(['index', 'coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'quote_id', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_id', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified',
       'is_misinfo', 'abbr', 'state', 'latitude', 'longitude'],
      dtype='object')

In [ ]:
comb_complete['latitude'].iloc[3]

39.055

In [ ]:
comb_complete['latitude'][3]

39.055

In [ ]:
for i in range(comb_complete.shape[0]):
  if isinstance(comb_complete['latitude'][i], float) == False:
    print(i, type(comb_complete['latitude'][i]))